### Visualizing the distribution of the observations

### Load the required libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index= 'BSBSSSBB'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print 'keys from STAT=',STAT.keys()

#read data
filename=data_dir+'/US_Weather_%s.parquet'%file_index

df=sqlContext.read.parquet(filename)
print df.count()
df.show(10)

Populating the interactive namespace from numpy and matplotlib
keys from STAT= ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP']
13081
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|     36.6| 31.9297| -82.6792|       TMAX|USC00095386|    14|[90 55 A0 56 38 5...|1946.0|BSBSSSBB|
|     36.6| 31.9297| -82.6792|       TMAX|USC00095386|     0|[B0 58 E0 55 B0 5...|1947.0|BSBSSSBB|
|     36.6| 31.9297| -82.6792|       TMAX|USC00095386|     1|[78 5B 98 5A A0 5...|1948.0|BSBSSSBB|
|     36.6| 31.9297| -82.6792|       TMAX|USC00095386|    32|[70 5A 98 5A 48 5...|1950.0|BSBSSSBB|
|     36.6| 31.9297| -82.6792|       TMAX|USC00095386|     7|[00 58 90 59 C8 5...|1951.0|BSBSSSBB|
|     36.6| 31.9297| -82.6792|       TMAX|USC00095386|     9|[D0 5B 14 5C 

### Select data for a particular station and measurement type

In [2]:
### Function to extract the values 
measures = ['TOBS', 'TMIN', 'TMAX', 'PRCP', 'SNOW', 'SNWD']
data = []
for i in measures:
    sqlContext.registerDataFrameAsTable(df,'weather')
    Query="SELECT station, elevation, latitude, longitude, year, vector FROM weather\n\tWHERE measurement='%s'"%(i)
    df1 = sqlContext.sql(Query)
    rows=df1.rdd.map(lambda row:[unpackArray(row['vector'],np.float16), row['station'],  row['elevation'], row['latitude'], row['longitude'], row['year']]).collect()
    measure = pd.DataFrame(rows, columns=('vector', 'station',  'elevation', 'latitude', 'longitude', 'year'))
    data.append(measure)

In [3]:
for i in range(0,4):
    data[i]['Jan_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[:31])/10)
    data[i]['Feb_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[31:59])/10)
    data[i]['Mar_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[59:90])/10)
    data[i]['Apr_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[90:120])/10)
    data[i]['May_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[120:151])/10)
    data[i]['Jun_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[151:181])/10)
    data[i]['Jul_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[181:212])/10)
    data[i]['Aug_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[212:243])/10)
    data[i]['Sep_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[243:273])/10)
    data[i]['Oct_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[273:304])/10)
    data[i]['Nov_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[304:334])/10)
    data[i]['Dec_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[334:])/10)
    data[i]['Annual_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x)/10)
    data[i] = data[i].groupby(['station',  'elevation', 'latitude', 'longitude', 'year']).mean()
for i in range(4,6):
    data[i]['Jan_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[:31]))
    data[i]['Feb_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[31:59]))
    data[i]['Mar_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[59:90]))
    data[i]['Apr_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[90:120]))
    data[i]['May_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[120:151]))
    data[i]['Jun_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[151:181]))
    data[i]['Jul_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[181:212]))
    data[i]['Aug_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[212:243]))
    data[i]['Sep_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[243:273]))
    data[i]['Oct_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[273:304]))
    data[i]['Nov_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[304:334]))
    data[i]['Dec_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x[334:]))
    data[i]['Annual_%s' %measures[i]] = data[i]['vector'].apply(lambda x: np.nanmean(x))
    data[i] = data[i].groupby(['station',  'elevation', 'latitude', 'longitude', 'year']).mean()

/root/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [4]:
print data[0].shape, data[1].shape, data[2].shape, data[3].shape, data[4].shape, data[5].shape

(1660, 13) (2203, 13) (2203, 13) (2833, 13) (2125, 13) (2057, 13)


In [5]:
data = pd.concat([data[0], data[1], data[2], data[3], data[4], data[5]], axis=1, join='outer')
print data.shape
data.head()

(3006, 78)


Jan_TOBS  Feb_TOBS  Mar_TOBS  \
station     elevation latitude longitude year                                   
US1FLBK0002 29.9      30.2447  -82.1224  2008.0       NaN       NaN       NaN   
US1FLBK0003 40.2      30.3212  -82.2757  2008.0       NaN       NaN       NaN   
                                         2009.0       NaN       NaN       NaN   
                                         2010.0       NaN       NaN       NaN   
                                         2011.0       NaN       NaN       NaN   

                                                 Apr_TOBS  May_TOBS  Jun_TOBS  \
station     elevation latitude longitude year                                   
US1FLBK0002 29.9      30.2447  -82.1224  2008.0       NaN       NaN       NaN   
US1FLBK0003 40.2      30.3212  -82.2757  2008.0       NaN       NaN       NaN   
                                         2009.0       NaN       NaN       NaN   
                                         2010.0       NaN       NaN       NaN   
                                         2011.0       NaN       NaN       NaN   

                                                 Jul_TOBS  Aug_TOBS  Sep_TOBS  \
station     elevation latitude longitude year                                   
US1FLBK0002 29.9      30.2447  -82.1224  2008.0       NaN       NaN       NaN   
US1FLBK0003 40.2      30.3212  -82.2757  2008.0       NaN       NaN       NaN   
                                         2009.0       NaN       NaN       NaN   
                                         2010.0       NaN       NaN       NaN   
                                         2011.0       NaN       NaN       NaN   

                                                 Oct_TOBS     ...       \
station     elevation latitude longitude year                 ...        
US1FLBK0002 29.9      30.2447  -82.1224  2008.0       NaN     ...        
US1FLBK0003 40.2      30.3212  -82.2757  2008.0       NaN     ...        
                                         2009.0       NaN     ...        
                                         2010.0       NaN     ...        
                                         2011.0       NaN     ...        

                                                 Apr_SNWD  May_SNWD  Jun_SNWD  \
station     elevation latitude longitude year                                   
US1FLBK0002 29.9      30.2447  -82.1224  2008.0       NaN       NaN       NaN   
US1FLBK0003 40.2      30.3212  -82.2757  2008.0       NaN       NaN       NaN   
                                         2009.0       NaN       NaN       NaN   
                                         2010.0       NaN       NaN       NaN   
                                         2011.0       NaN       NaN       NaN   

                                                 Jul_SNWD  Aug_SNWD  Sep_SNWD  \
station     elevation latitude longitude year                                   
US1FLBK0002 29.9      30.2447  -82.1224  2008.0       NaN       NaN       NaN   
US1FLBK0003 40.2      30.3212  -82.2757  2008.0       NaN       NaN       NaN   
                                         2009.0       NaN       NaN       NaN   
                                         2010.0       NaN       NaN       NaN   
                                         2011.0       NaN       NaN       NaN   

                                                 Oct_SNWD  Nov_SNWD  Dec_SNWD  \
station     elevation latitude longitude year                                   
US1FLBK0002 29.9      30.2447  -82.1224  2008.0       NaN       NaN       NaN   
US1FLBK0003 40.2      30.3212  -82.2757  2008.0       NaN       NaN       NaN   
                                         2009.0       NaN       NaN       NaN   
                                         2010.0       NaN       NaN       NaN   
                                         2011.0       NaN       NaN       NaN   

                                                 Annual_SNWD  
station     elevation latitude longitude year              

In [6]:
data.to_csv('./to_tableau.csv')